In [20]:
import os
from pathlib import Path
import yaml
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
def get_project_root():
    # Assuming a known file (e.g., 'main.py') exists in the project root
    # Adjust this to a file that reliably exists at your project root
    current_file_path = Path("../data/processed/data.yml").resolve() 
    
    # Traverse upwards until a known project root indicator is found
    # For example, searching for a '.git' directory
    for parent in current_file_path.parents:
        if (parent / ".git").exists():
            return parent
    
    # Fallback or error handling if root cannot be found
    return current_file_path.parent # Or raise an error

project_root = get_project_root()
print(f"Project Root: {project_root}")

Project Root: /Users/jennarong/Repos/Driver-drowsiness-detection


In [22]:
# โหลด config จาก data.yml
with open(project_root / "data/processed/data.yaml", "r") as f:
    data_cfg = yaml.safe_load(f)

train_dir = project_root / "data/processed/" / data_cfg["train"]
val_dir = project_root / "data/processed/" / data_cfg["val"]
test_dir = project_root / "data/processed/" / data_cfg["test"]
class_names = data_cfg.get("names", None)  # optional
num_classes = data_cfg.get("nc", None)

print(f"train path: {train_dir}");
print(f"val path: {val_dir}");
print(f"test path: {test_dir}");
print(f"class_names: {class_names}");
print(f"num_classes: {num_classes}");

train path: /Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/train/images
val path: /Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/valid/images
test path: /Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/train/images
class_names: ['Microsleep', 'Nosleepiness', 'yawn']
num_classes: 3


In [47]:
IMG_SIZE = (224, 224)

def load_dataset(images_dir, labels_dir, num_classes):
    image_paths, labels = [], []

    for fname in os.listdir(images_dir):
        if fname.endswith(".jpg") or fname.endswith(".png"):
            img_path = os.path.join(images_dir, fname)
            label_path = os.path.join(labels_dir, fname.replace(".jpg", ".txt").replace(".png", ".txt"))
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    readline = f.readline()
                    if readline.strip() == "":
                        class_id = 0   # default
                    else:
                        readline = readline.split(" ")
                        # print(f"{label_path} {readline}")
                        class_id = int(readline[0])  # just take the class id
                    
                image_paths.append(img_path)
                labels.append(class_id)

    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    def process(img_path, label):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, IMG_SIZE)
        img = img / 255.0
        return img, tf.one_hot(label, num_classes)

    return ds.map(process).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

In [48]:
NUM_CLASSES = num_classes   # 🔧 change to your actual number of classes
train_ds = load_dataset(train_dir, project_root / "data/processed/train/labels", NUM_CLASSES)
val_ds   = load_dataset(val_dir, project_root / "data/processed/valid/labels", NUM_CLASSES)

/Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/train/labels/22-09-23_mp4-1976_jpg.rf.a75a760fb5e574b36c141e7f1f10b75e.txt ['2', '0.41532291666666665', '0.42837037037037035', '0.22077604166666673', '0.6758148148148148']
/Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/train/labels/WIN_20231015_15_56_51_Pro_mp4-47_jpg.rf.c7eeb9d261d9d7d62128ef4c74df33d6.txt ['1', '0.515234375', '0.44722222222222224', '0.28515625', '0.6166666666666667']
/Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/train/labels/2023-09-22-12-05-19_mp4-309_jpg.rf.6ac3188f6d97d00eac70d725f16ab219.txt ['0', '0.527734375', '0.5513888888888889', '0.47109375', '0.8972222222222223']
/Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/train/labels/P1042762_720_mp4-1176_jpg.rf.fee70f008ad418007388a7d7a5bbe2f6.txt ['1', '0.68046875', '0.3715277777777778', '0.21015625', '0.4513888888888889']
/Users/jennarong/Repos/Driver-drowsiness-detection/data/processed/train/l

In [49]:
# โหลด VGG16 (ไม่เอา layer ข้างบน)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [50]:
# Freeze บางเลเยอร์ (เพื่อใช้ feature extractor เดิม)
for layer in base_model.layers:
    layer.trainable = False

In [51]:
# เพิ่มเลเยอร์ด้านบน (custom classifier)
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [52]:
model = Model(inputs=base_model.input, outputs=predictions)

In [53]:
# Compile โมเดล
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,138,243 (80.64 MB)

 Trainable params: 6,423,555 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [55]:
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
 11/392 ━━━━━━━━━━━━━━━━━━━━ 24:59 4s/step - accuracy: 0.4591 - loss: 1.1972

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107ad9b80>>
Traceback (most recent call last):
  File "/Users/jennarong/Library/Python/3.9/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# model.save("vgg16_classifier.h5")